#### Copyright (C) 2017 The University of Sydney, Australia
This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program; if not, write to Free Software Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301, USA.

Author:  John Cannon

## Calculate average seafloor spreading rates of mid-ocean ridges over a sliding time window (over time)

The output file contains the following columns:

     reconstruction_time (Ma)
     total ridge length in metres (excluding transform segments)
     average spreading rate in cm/year (over 'averaging_time_window' million years)
     standard deviation of spreading rate in cm/year (over 'averaging_time_window' million years)

The spreading rates are only calculated along ridge sections (transform sections are excluded). Note that this algorithm only works well under the following conditions:

* ridge segments are perpendicular to their spreading directions
* isochron geometries are up-to-date with respect to the rotation model
  * ie, stage pole is in correct location relative to geometry
* there are valid rotations (in rotation model) for each isochron at its birth time plus one
  * ie, 1My prior to isochron birth time
* all isochrons have conjugate plate IDs

In [ ]:
from __future__ import print_function
import math
import os
import pygplates
import sys
from ptt import ridge_spreading_rate

# Input rotation and topology files.
rotation_filename = '../data/Global_EarthByte_230-0Ma_GK07_AREPS.rot'
rotation_model = pygplates.RotationModel(rotation_filename)
topology_filenames = [
        '../data/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',
        '../data/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']

# Output file containing results at each reconstruction time.
output_filename = '../data/ridge_spreading_rates.txt'

# The time window over which to average spreading rates (in My).
averaging_time_window = 1

# Define the time range.
# The reconstruction time range to output statistics for.
#
# NOTE: Topologies are resolved up to time 'max_time + averaging_time_window - 1'
#       so can average over past 'averaging_time_window' million years.
min_time = 0
max_time = 230

# Tessellate the subduction zones to 0.5 degrees.
threshold_sampling_distance_radians = math.radians(0.5)

# List of spreading feature types to consider.
# Only look at mid-ocean ridge features.
spreading_feature_types = [pygplates.FeatureType.gpml_mid_ocean_ridge]
# Consider all 'spreading' features.
# A spreading feature has left/right plate IDs (eg, mid-ocean ridge) or
# conjugate plate IDs (eg, isochron).
#spreading_feature_types = None

In [ ]:
print('Calculating spreading rates from {0} to {1} Ma...'.format(min_time, max_time))

# Each element of this list will be a tuple of values for a specific reconstruction time.
output_data = []

# Read the topology filenames once instead of at every time step.
topology_features = []
for topology_filename in topology_filenames:
    topology_features.extend(pygplates.FeatureCollection(topology_filename))

# Keep track of spreading-rate-related quantities over time so can later calculate statistics.
total_length_metres_time_sequence = []
total_spreading_rate_times_length_time_sequence = []
total_spreading_rate_squared_times_length_time_sequence = []

# Iterate over times in inclusive range [min_time, max_time + averaging_time_window - 1].
for reconstruction_time in range(min_time, max_time + averaging_time_window):
    
    # Use existing ridge spreading rate script to generate sample points along mid-ocean ridges at 'time'.
    ridge_spreading_rate_data = ridge_spreading_rate.spreading_rates(
        rotation_model,
        topology_features,
        reconstruction_time,
        threshold_sampling_distance_radians,
        spreading_feature_types)
    
    #
    # Calculate average seafloor spreading for the current reconstruction time.
    #
    total_length_metres = 0.0
    total_spreading_rate_times_length = 0.0
    total_spreading_rate_squared_times_length = 0.0
    for point_lon, point_lat, spreading_rate, length_degrees in ridge_spreading_rate_data:
        
        length_metres = math.radians(length_degrees) * 1e3 * pygplates.Earth.mean_radius_in_kms
        
        total_length_metres += length_metres
        total_spreading_rate_times_length += spreading_rate * length_metres
        total_spreading_rate_squared_times_length += spreading_rate * spreading_rate * length_metres
    
    total_length_metres_time_sequence.append(total_length_metres)
    total_spreading_rate_times_length_time_sequence.append(total_spreading_rate_times_length)
    total_spreading_rate_squared_times_length_time_sequence.append(total_spreading_rate_squared_times_length)
    

#
# Statistics.
#
# mean = M = sum(Ci * Xi) / sum(Ci)
# std_dev  = sqrt[sum(Ci * (Xi - M)^2) / sum(Ci)]
#          = sqrt[(sum(Ci * Xi^2) - 2 * M * sum(Ci * Xi) + M^2 * sum(Ci)) / sum(Ci)]
#          = sqrt[(sum(Ci * Xi^2) - 2 * M * M * sum(Ci) + M^2 * sum(Ci)) / sum(Ci)]
#          = sqrt[(sum(Ci * Xi^2) - M^2 * sum(Ci)) / sum(Ci)]
#          = sqrt[(sum(Ci * Xi^2) / sum(Ci) - M^2]
#
# ...where N is total number of sample points, Xi are sample and Ci are weights.
#

# Iterate over times in inclusive range [min_time, max_time].
for reconstruction_time_index, reconstruction_time in enumerate(range(min_time, max_time + 1)):
    
    total_length_metres = 0
    total_spreading_rate_times_length = 0.0
    total_spreading_rate_squared_times_length = 0.0
    
    # Average over the time window (of 'averaging_time_window' million years).
    for time_index in range(reconstruction_time_index, reconstruction_time_index + averaging_time_window):
        total_length_metres += total_length_metres_time_sequence[time_index]
        total_spreading_rate_times_length += total_spreading_rate_times_length_time_sequence[time_index]
        total_spreading_rate_squared_times_length += total_spreading_rate_squared_times_length_time_sequence[time_index]
    
    mean_spreading_rate = total_spreading_rate_times_length / total_length_metres
    variance_spreading_rate = (total_spreading_rate_squared_times_length / total_length_metres -
                                       mean_spreading_rate * mean_spreading_rate)
    std_dev_spreading_rate = (math.sqrt(variance_spreading_rate)
                                      if variance_spreading_rate > 0.0 else 0.0)
    
    # Add a tuple of data values.
    # Can be any number of elements (they'll get written as columns to the output text file in the same order).
    output_data.append((
            reconstruction_time,
            # We want the total length at the reconstruction time (not summed over averaging time window)...
            total_length_metres_time_sequence[reconstruction_time_index],
            mean_spreading_rate,
            std_dev_spreading_rate))

In [ ]:
print('Writing average spreading rates to "{0}"...'.format(output_filename))

# Write the output data to a text file.
with open(output_filename, 'w') as output_file:
    for output_line in output_data:
        # Each 'output_line' is a tuple of values for a specific reconstruction time.
        # Write the elements of the tuple separated by spaces (and ending with a newline).
        output_file.write(' '.join(str(item) for item in output_line) + '\n')

print('...finished.')